In [0]:
!pip uninstall tensorflow
!pip install tensorflow-gpu

In [0]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Input

In [0]:
input_layer = Input((128, 128, 3))

In [0]:
body = MobileNet(include_top=False, input_tensor=input_layer, input_shape=None, pooling=None, classes=2)

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [0]:
body.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32) 

In [0]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model

In [0]:
x = Flatten()(body.output)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(128)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(2, activation='softmax')(x)

In [0]:
model = Model(body.input, x)

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128 

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/Colab Notebooks"

In [0]:
import os
from random import shuffle
import cv2
import numpy as np

In [0]:
bee_a_dirs = os.listdir('MARDI-dataset/cropped/bee-A/')
bee_b_dirs = os.listdir('MARDI-dataset/cropped/bee-B/')

In [0]:
bee_a_dirs_map_c = map(lambda x: (x, 0), bee_a_dirs)
bee_b_dirs_map_c = map(lambda x: (x, 1), bee_b_dirs)

In [0]:
Xdir_y = list(bee_b_dirs_map_c) + list(bee_a_dirs_map_c)

In [0]:
shuffle(Xdir_y)
len(Xdir_y)

6551

In [0]:
def generate_Xy(imgs_generated):
    X_train = []
    y_train = []
    for img in imgs_generated:
        if img[1] == 0:
            raw_img = cv2.imread('MARDI-dataset/cropped/bee-A/' + img[0])
            raw_img = cv2.resize(raw_img, (128, 128))
            y_train.append([1, 0])
        else:
            raw_img = cv2.imread('MARDI-dataset/cropped/bee-B/' + img[0])
            raw_img = cv2.resize(raw_img, (128, 128))
            y_train.append([0, 1])
        X_train.append(raw_img)
        
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    return X_train, y_train
    

def set_generator(Xdir_y, batch_size):
		pointer = 0
		while True:
			imgs_generated = []
			for i in range(batch_size):
				if pointer == len(Xdir_y):
					pointer = 0
				imgs_generated.append(Xdir_y[pointer])
				pointer += 1
			X_train, y_train = generate_Xy(imgs_generated)
			yield (X_train, y_train)
            
def split_list(a_list):
    seven_three = len(a_list)//10 * 7
    return a_list[:seven_three], a_list[seven_three:]

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

model.compile(optimizer=Adam(), metrics=['accuracy'], loss=categorical_crossentropy)

Xdir_y_train, Xdir_y_test = split_list(Xdir_y)

train_gen = set_generator(Xdir_y_train, batch_size=32)
test_gen = set_generator(Xdir_y_test, batch_size=32)

In [0]:
print(len(Xdir_y_train), len(Xdir_y_test))

4585 1966


In [0]:
!nvidia-smi

Sun Oct 27 10:13:49 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    70W / 149W |    163MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



In [0]:
model.fit_generator(train_gen, 
                    steps_per_epoch=len(Xdir_y_train)//32,  
                    validation_data=test_gen, 
                    validation_steps=len(Xdir_y_test)//32,
                    verbose=1,
                    epochs=5
                   )

Epoch 1/5
143/143 [==============================] - 49s 344ms/step - loss: 0.3165 - accuracy: 0.9482 - val_loss: 0.1558 - val_accuracy: 0.9718
Epoch 2/5
143/143 [==============================] - 48s 337ms/step - loss: 0.0704 - accuracy: 0.9823 - val_loss: 0.2861 - val_accuracy: 0.9575
Epoch 3/5
143/143 [==============================] - 48s 338ms/step - loss: 0.0349 - accuracy: 0.9886 - val_loss: 0.1096 - val_accuracy: 0.9816
Epoch 4/5
143/143 [==============================] - 48s 336ms/step - loss: 0.0378 - accuracy: 0.9886 - val_loss: 0.1086 - val_accuracy: 0.9816
Epoch 5/5
143/143 [==============================] - 49s 340ms/step - loss: 0.0256 - accuracy: 0.9904 - val_loss: 0.2031 - val_accuracy: 0.9805


In [0]:
model.save('mobilenet-2class-bees-model.h5')
model.save_weights('mobilenet-2class-bees-weights.h5')